# Data Prep - Feature Engg

In [1]:
# %config IPCompleter.greedy=True
import pandas as pd
import numpy as np
import time

In [2]:
tt = pd.read_csv('data/test_QyjYwdj.csv')
tt['redemption_status'] = -1
tt['flag'] = 'Test'

print(tt.shape)
tt.head(2)

(50226, 6)


,id,campaign_id,coupon_id,customer_id,redemption_status,flag
0,3,22,869,967,-1,Test
1,4,20,389,1566,-1,Test


In [3]:
dt = pd.read_csv('data/train_AUpWtIz/train.csv')
print(dt.shape)
print(dt['redemption_status'].value_counts(normalize = True))
dt['flag'] = 'Train'
dt.head(2)

(78369, 5)
0    0.990698
1    0.009302
Name: redemption_status, dtype: float64


,id,campaign_id,coupon_id,customer_id,redemption_status,flag
0,1,13,27,1053,0,Train
1,2,13,116,48,0,Train


In [4]:
data = pd.concat([dt, tt])
print(data.shape)
data.head(2)

(128595, 6)


,id,campaign_id,coupon_id,customer_id,redemption_status,flag
0,1,13,27,1053,0,Train
1,2,13,116,48,0,Train


### Campaign and Customer Demographic Features

In [5]:
camp = pd.read_csv('data/train_AUpWtIz/campaign_data.csv')

camp['start_date']= pd.to_datetime(camp['start_date'], dayfirst=True) 
camp['end_date']= pd.to_datetime(camp['end_date'], dayfirst=True) 
print(camp.query('start_date > end_date').shape)

print(camp.shape)
camp.head(2)

(0, 4)
(28, 4)


,campaign_id,campaign_type,start_date,end_date
0,24,Y,2013-10-21,2013-12-20
1,25,Y,2013-10-21,2013-11-22


In [6]:
cust_d = pd.read_csv('data/train_AUpWtIz/customer_demographics.csv')
print(cust_d.shape)
cust_d.head(2)

(760, 7)


,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5


In [7]:
cust_d.isnull().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [8]:
cust_d["marital_status"] = cust_d["marital_status"].fillna("Unknown") 
cust_d["no_of_children"] = cust_d["no_of_children"].fillna("Unknown") 

In [9]:
cust_d.isnull().sum()

customer_id       0
age_range         0
marital_status    0
rented            0
family_size       0
no_of_children    0
income_bracket    0
dtype: int64

In [10]:
data = pd.merge(data, camp, how = 'left')
data = pd.merge(data, cust_d, how = 'left')

In [11]:
print(data.shape)
data.head(2)

(128595, 15)


,id,campaign_id,coupon_id,customer_id,redemption_status,flag,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,13,27,1053,0,Train,X,2013-05-19,2013-07-05,46-55,Unknown,0.0,1,Unknown,5.0
1,2,13,116,48,0,Train,X,2013-05-19,2013-07-05,36-45,Married,0.0,2,Unknown,3.0


In [12]:
data['campaign_len'] = data['end_date'] - data['start_date']
data['campaign_len'] = data['campaign_len'].apply(lambda x : x.days)
data['campaign_len'].describe()

count    128595.000000
mean         45.907586
std           9.665459
min          32.000000
25%          38.000000
50%          47.000000
75%          53.000000
max          70.000000
Name: campaign_len, dtype: float64

In [13]:
print(data['start_date'].min())
print(data['start_date'].max())

2012-08-12 00:00:00
2013-10-21 00:00:00


In [14]:

data['camp_months'] = data['start_date'].apply(lambda x : str(x.month)) + '-' + data['end_date'].apply(lambda x : str(x.month))


In [15]:
# data['camp_year'] = data['start_date'].apply(lambda x : str(x.year))
# data['camp_year'].value_counts()

In [16]:
data[data['redemption_status'] == 0]['camp_months'].value_counts(normalize = True)

2-4      0.287944
5-7      0.286708
11-1     0.082792
8-9      0.077331
4-5      0.054791
10-11    0.049961
3-4      0.040507
1-2      0.035871
4-6      0.035755
2-3      0.020350
12-1     0.011656
9-11     0.006118
12-2     0.005229
8-10     0.004160
1-3      0.000824
Name: camp_months, dtype: float64

In [17]:
data[data['redemption_status'] == 1]['camp_months'].value_counts(normalize = True)

5-7      0.474623
2-4      0.272977
8-9      0.071331
11-1     0.053498
3-4      0.031550
4-5      0.026063
10-11    0.021948
1-2      0.019204
12-1     0.008230
4-6      0.008230
2-3      0.005487
12-2     0.002743
8-10     0.001372
1-3      0.001372
9-11     0.001372
Name: camp_months, dtype: float64

In [18]:
data['camp_start_day'] = data['start_date'].apply(lambda x : str(x.dayofweek))

In [19]:
data[data['redemption_status'] == 0]['camp_start_day'].value_counts(normalize = True)

6    0.370157
5    0.341126
0    0.286888
2    0.001829
Name: camp_start_day, dtype: float64

In [20]:
data[data['redemption_status'] == 1]['camp_start_day'].value_counts(normalize = True)

6    0.547325
5    0.292181
0    0.159122
2    0.001372
Name: camp_start_day, dtype: float64

### Coupon Features

In [21]:
coup = pd.read_csv('data/train_AUpWtIz/coupon_item_mapping.csv')
print(coup.shape)
coup.head(2)

(92663, 2)


,coupon_id,item_id
0,105,37
1,107,75


In [22]:
item = pd.read_csv('data/train_AUpWtIz/item_data.csv')
print(item.shape)
item.head()

(74066, 4)


,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [23]:
coup = pd.merge(coup, item, how = 'left')
print(coup.shape)
coup.head(2)

(92663, 5)


,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,107,75,56,Local,Grocery


In [24]:
# coup.isnull().sum()

In [25]:
# coup.groupby('coupon_id')['brand_type'].apply(set).reset_index()

In [26]:
coup_features = coup.groupby('coupon_id').agg(unique_items = ('item_id', 'nunique'),
                                              unique_brands = ('brand', 'nunique'),
                                              brands = ('brand', set),
                                              brand_types = ('brand_type', set),
                                              categories = ('category', set)
                                             ).reset_index()
coup_features.head(2)

,coupon_id,unique_items,unique_brands,brands,brand_types,categories
0,1,39,3,"{1475, 4700, 1558}",{Established},"{Grocery, Natural Products}"
1,2,2,1,{2084},{Established},{Grocery}


In [27]:
coup_features['brands'] = coup_features['brands'].astype(str)
coup_features['brand_types'] = coup_features['brand_types'].astype(str)
coup_features['categories'] = coup_features['categories'].astype(str)

In [28]:
print(coup_features['brands'].nunique())
print(coup_features['brand_types'].nunique())
print(coup_features['categories'].nunique())

407
3
36


In [29]:
data = pd.merge(data, coup_features, how = 'left')

In [30]:
data["age_range"] = data["age_range"].fillna("Unknown")
data["marital_status"] = data["marital_status"].fillna("Unknown") 
data["rented"] = data["rented"].fillna("Unknown") 
data["family_size"] = data["family_size"].fillna("Unknown") 
data["no_of_children"] = data["no_of_children"].fillna("Unknown") 
data["income_bracket"] = data["income_bracket"].fillna("Unknown") 

In [31]:
data.isnull().sum()

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
flag                 0
campaign_type        0
start_date           0
end_date             0
age_range            0
marital_status       0
rented               0
family_size          0
no_of_children       0
income_bracket       0
campaign_len         0
camp_months          0
camp_start_day       0
unique_items         0
unique_brands        0
brands               0
brand_types          0
categories           0
dtype: int64

In [32]:
print(data.shape)
data.head(2)

(128595, 23)


,id,campaign_id,coupon_id,customer_id,redemption_status,flag,campaign_type,start_date,end_date,age_range,...,no_of_children,income_bracket,campaign_len,camp_months,camp_start_day,unique_items,unique_brands,brands,brand_types,categories
0,1,13,27,1053,0,Train,X,2013-05-19,2013-07-05,46-55,...,Unknown,5,47,5-7,6,125,2,"{1105, 1636}",{'Established'},{'Grocery'}
1,2,13,116,48,0,Train,X,2013-05-19,2013-07-05,36-45,...,Unknown,3,47,5-7,6,3,1,{56},{'Local'},{'Grocery'}


##### Brand Analysis

In [33]:

# pd.DataFrame(data['brands'].value_counts()).sort_values(by = 'brands', ascending = False) #.to_clipboard()


In [34]:

# pd.DataFrame(data[data['redemption_status'] == 1]['brands'].value_counts()).sort_values(by = 'brands', ascending = False).head(20)  


### Transaction Info

In [35]:
cust_t = pd.read_csv('data/train_AUpWtIz/customer_transaction_data.csv')
cust_t['date'] = pd.to_datetime(cust_t['date'])
print(cust_t.shape)
cust_t.head(2)

(1324566, 7)


,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0


In [36]:
cust_t = pd.merge(cust_t, item, how = 'left')
print(cust_t.shape)
cust_t.head(2)

(1324566, 10)


,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products


In [37]:
def f(row):
    if row['brand_type'] == 'Established':
        val = 1
    else:
        val = 0
    return val

cust_t['established_flag'] = cust_t.apply(f, axis = 1)

print(cust_t.shape)
cust_t.head(2)

(1324566, 11)


,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,established_flag
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products,0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products,0


In [38]:
cust_t['established_qty'] = cust_t['established_flag'] * cust_t['quantity']

In [39]:
cust_basket = cust_t.groupby(['customer_id']).agg(cust_basket = ('item_id', set),
                                                  cust_basket_size = ('item_id', 'nunique'),
                                                  cust_catg_basket = ('category', set),
                                                  cust_brand_basket = ('brand', set),                                                  
                                                  cust_brand_type_basket = ('brand_type', set),
                                                  established_perc = ('established_flag', 'mean'),
                                                  established_qty_mean = ('established_qty', 'mean')
                                                 ).reset_index()
cust_basket['key'] = 1
print(cust_basket.shape)
cust_basket.head(2)

(1582, 9)


,customer_id,cust_basket,cust_basket_size,cust_catg_basket,cust_brand_basket,cust_brand_type_basket,established_perc,established_qty_mean,key
0,1,"{49158, 45063, 10249, 61452, 18449, 45074, 676...",463,"{Prepared Food, Grocery, Packaged Meat, Meat, ...","{1, 1538, 1025, 4615, 522, 11, 524, 2065, 531,...","{Established, Local}",0.914122,1.081107,1
1,2,"{22528, 16386, 24587, 61453, 59408, 20502, 164...",352,"{Grocery, Dairy, Juices & Snacks, Packaged Mea...","{1, 514, 1025, 5125, 522, 11, 5131, 524, 1041,...","{Established, Local}",0.780430,0.890215,1


In [40]:
# cust_basket['cust_brand_type_basket'].value_counts(normalize = True)
# cust_basket['established_perc'].describe()

In [41]:
coup_basket = coup.groupby(['coupon_id']).agg(coup_basket = ('item_id', set),
                                              coup_basket_size = ('item_id', 'nunique'),
                                              coup_catg_basket = ('category', set),
                                              coup_brand_basket = ('brand', set),
                                              coup_brand_type_basket = ('brand_type', set)
                                             ).reset_index()

coup_basket['key'] = 1
print(coup_basket.shape)
coup_basket.head(2)

(1116, 7)


,coupon_id,coup_basket,coup_basket_size,coup_catg_basket,coup_brand_basket,coup_brand_type_basket,key
0,1,"{50199, 60068, 59942, 4007, 4140, 44076, 43951...",39,"{Grocery, Natural Products}","{1475, 4700, 1558}",{Established},1
1,2,"{2581, 12901}",2,{Grocery},{2084},{Established},1


In [42]:
big = pd.merge(cust_basket,coup_basket, on='key').drop('key', axis=1)
print(big.shape)
big.head(2)

(1765512, 14)


,customer_id,cust_basket,cust_basket_size,cust_catg_basket,cust_brand_basket,cust_brand_type_basket,established_perc,established_qty_mean,coupon_id,coup_basket,coup_basket_size,coup_catg_basket,coup_brand_basket,coup_brand_type_basket
0,1,"{49158, 45063, 10249, 61452, 18449, 45074, 676...",463,"{Prepared Food, Grocery, Packaged Meat, Meat, ...","{1, 1538, 1025, 4615, 522, 11, 524, 2065, 531,...","{Established, Local}",0.914122,1.081107,1,"{50199, 60068, 59942, 4007, 4140, 44076, 43951...",39,"{Grocery, Natural Products}","{1475, 4700, 1558}",{Established}
1,1,"{49158, 45063, 10249, 61452, 18449, 45074, 676...",463,"{Prepared Food, Grocery, Packaged Meat, Meat, ...","{1, 1538, 1025, 4615, 522, 11, 524, 2065, 531,...","{Established, Local}",0.914122,1.081107,2,"{2581, 12901}",2,{Grocery},{2084},{Established}


In [43]:
data = pd.merge(data, big, how = 'left')
print(data.shape)
data.head(2)

(128595, 35)


,id,campaign_id,coupon_id,customer_id,redemption_status,flag,campaign_type,start_date,end_date,age_range,...,cust_catg_basket,cust_brand_basket,cust_brand_type_basket,established_perc,established_qty_mean,coup_basket,coup_basket_size,coup_catg_basket,coup_brand_basket,coup_brand_type_basket
0,1,13,27,1053,0,Train,X,2013-05-19,2013-07-05,46-55,...,"{Prepared Food, Grocery, Packaged Meat, Seafoo...","{2048, 1, 1409, 771, 644, 136, 5129, 910, 1294...","{Established, Local}",0.535484,0.719355,"{3076, 3077, 7176, 1544, 34314, 72713, 64521, ...",125,{Grocery},"{1105, 1636}",{Established}
1,2,13,116,48,0,Train,X,2013-05-19,2013-07-05,36-45,...,"{Prepared Food, Grocery, Dairy, Juices & Snack...","{1, 522, 11, 524, 2580, 1556, 1558, 544, 5154,...","{Established, Local}",0.755844,1.122078,"{36721, 4157, 39581}",3,{Grocery},{56},{Local}


In [44]:
data.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'flag', 'campaign_type', 'start_date', 'end_date', 'age_range',
       'marital_status', 'rented', 'family_size', 'no_of_children',
       'income_bracket', 'campaign_len', 'camp_months', 'camp_start_day',
       'unique_items', 'unique_brands', 'brands', 'brand_types', 'categories',
       'cust_basket', 'cust_basket_size', 'cust_catg_basket',
       'cust_brand_basket', 'cust_brand_type_basket', 'established_perc',
       'established_qty_mean', 'coup_basket', 'coup_basket_size',
       'coup_catg_basket', 'coup_brand_basket', 'coup_brand_type_basket'],
      dtype='object')

In [45]:
data.groupby('redemption_status')['established_qty_mean'].describe(percentiles = [x/100 for x in list(range(10,100,10))])

,count,mean,std,min,10%,20%,30%,40%,50%,60%,70%,80%,90%,max
redemption_status,,,,,,,,,,,,,,
-1,50226.0,0.958455,0.189412,0.416493,0.751553,0.824554,0.870968,0.910215,0.954103,0.988889,1.032995,1.080229,1.149979,4.245179
0,77640.0,0.960801,0.211196,0.416493,0.744113,0.817216,0.868732,0.908560,0.949861,0.989583,1.033789,1.087087,1.175644,4.245179
1,729.0,0.964935,0.168658,0.537984,0.745116,0.840370,0.898477,0.945722,0.982363,1.014488,1.042036,1.064010,1.122340,2.004751


In [46]:
data['ovr'] = -999
data['over_set'] = 0

start = time.time()
for i in range(len(data)):
    overlap = data['coup_basket'].iloc[i] & data['cust_basket'].iloc[i]
    data['over_set'].iloc[i] = overlap
    data['ovr'].iloc[i] = len(overlap)

end = time.time()
print(end-start)

/Users/a0t00ir/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


842.6055128574371


In [47]:
data['perc_ovr'] = data['ovr']/data['cust_basket_size']

In [48]:
print(data.shape)
data.head(2)

(128595, 38)


,id,campaign_id,coupon_id,customer_id,redemption_status,flag,campaign_type,start_date,end_date,age_range,...,established_perc,established_qty_mean,coup_basket,coup_basket_size,coup_catg_basket,coup_brand_basket,coup_brand_type_basket,ovr,over_set,perc_ovr
0,1,13,27,1053,0,Train,X,2013-05-19,2013-07-05,46-55,...,0.535484,0.719355,"{3076, 3077, 7176, 1544, 34314, 72713, 64521, ...",125,{Grocery},"{1105, 1636}",{Established},0,{},0.0
1,2,13,116,48,0,Train,X,2013-05-19,2013-07-05,36-45,...,0.755844,1.122078,"{36721, 4157, 39581}",3,{Grocery},{56},{Local},0,{},0.0


In [49]:
data.isnull().sum()

id                        0
campaign_id               0
coupon_id                 0
customer_id               0
redemption_status         0
flag                      0
campaign_type             0
start_date                0
end_date                  0
age_range                 0
marital_status            0
rented                    0
family_size               0
no_of_children            0
income_bracket            0
campaign_len              0
camp_months               0
camp_start_day            0
unique_items              0
unique_brands             0
brands                    0
brand_types               0
categories                0
cust_basket               0
cust_basket_size          0
cust_catg_basket          0
cust_brand_basket         0
cust_brand_type_basket    0
established_perc          0
established_qty_mean      0
coup_basket               0
coup_basket_size          0
coup_catg_basket          0
coup_brand_basket         0
coup_brand_type_basket    0
ovr                 

### Transaction Info - 2.0

In [50]:
cust_t = pd.read_csv('data/train_AUpWtIz/customer_transaction_data.csv')
cust_t['date'] = pd.to_datetime(cust_t['date'])
print(cust_t.shape)
print(cust_t['date'].min())
print(cust_t['date'].max())
cust_t.head(2)

(1324566, 7)
2012-01-02 00:00:00
2013-07-03 00:00:00


,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0


In [51]:
camp = pd.read_csv('data/train_AUpWtIz/campaign_data.csv')
camp['start_date']= pd.to_datetime(camp['start_date'], dayfirst=True) 
camp['end_date']= pd.to_datetime(camp['end_date'], dayfirst=True) 

dates = pd.DataFrame(pd.date_range(start = camp['start_date'].min(),end = camp['end_date'].max()), columns=['date'])
camp2 = pd.merge(left=dates, right=camp, left_on='date', right_on='start_date', how='outer').fillna(method='ffill')
print(camp2.shape)
print(camp2['start_date'].min())
print(camp2['end_date'].max())
camp2.head(2)

(499, 5)
2012-08-12 00:00:00
2013-12-20 00:00:00


,date,campaign_id,campaign_type,start_date,end_date
0,2012-08-12,26.0,X,2012-08-12,2012-09-21
1,2012-08-13,26.0,X,2012-08-12,2012-09-21


In [52]:
cust_t2 = pd.merge(cust_t, camp2, how = 'left')

In [53]:
cust_t2

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,campaign_id,campaign_type,start_date,end_date
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,NaN,NaN,NaT,NaT
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,NaN,NaN,NaT,NaT
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,NaN,NaN,NaT,NaT
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,NaN,NaN,NaT,NaT
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...
1326951,2013-06-30,1129,2777,1,284.60,-71.24,0.0,13.0,X,2013-05-19,2013-07-05
1326952,2013-06-30,1129,2953,4,42.74,-28.50,0.0,13.0,X,2013-05-19,2013-07-05
1326953,2013-06-30,1129,2971,6,64.12,-42.74,0.0,13.0,X,2013-05-19,2013-07-05
1326954,2013-06-30,1129,46984,1,95.82,0.00,0.0,13.0,X,2013-05-19,2013-07-05


In [54]:
cust_t2['campaign_id'] = cust_t2['campaign_id'].fillna(value = 0)

def f(row):
    if row['campaign_id'] == 0:
        val = 0
    else:
        val = 1
    return val

cust_t2['campaign_id'] = cust_t2.apply(f , axis = 1)

cust_t2

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,campaign_id,campaign_type,start_date,end_date
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,0,NaN,NaT,NaT
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,0,NaN,NaT,NaT
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,0,NaN,NaT,NaT
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,0,NaN,NaT,NaT
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,0,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...
1326951,2013-06-30,1129,2777,1,284.60,-71.24,0.0,1,X,2013-05-19,2013-07-05
1326952,2013-06-30,1129,2953,4,42.74,-28.50,0.0,1,X,2013-05-19,2013-07-05
1326953,2013-06-30,1129,2971,6,64.12,-42.74,0.0,1,X,2013-05-19,2013-07-05
1326954,2013-06-30,1129,46984,1,95.82,0.00,0.0,1,X,2013-05-19,2013-07-05


In [55]:
cust_t2['camp_quantity'] = cust_t2['quantity'] * cust_t2['campaign_id']

cust = cust_t2.groupby(['customer_id']).agg(camp_cnt = ('campaign_id', 'sum'),
                                            cnt2 = ('campaign_id','count'),
                                            camp_qty = ('camp_quantity','sum'),
                                            qty2 = ('quantity','sum')
                                           ).reset_index()

cust['camp_shop_perc'] = cust['camp_cnt']/cust['cnt2']
cust['camp_qty_perc'] = cust['camp_qty']/cust['qty2']

# cust['camp_shop_perc'].describe(percentiles = [x/100 for x in list(range(10,100,10))])

cust

,customer_id,camp_cnt,cnt2,camp_qty,qty2,camp_shop_perc,camp_qty_perc
0,1,731,1048,845,1227,0.697519,0.688672
1,2,256,419,288,474,0.610979,0.607595
2,3,450,717,7728,8184,0.627615,0.944282
3,4,102,220,126,280,0.463636,0.450000
4,5,533,792,93034,93353,0.672980,0.996583
...,...,...,...,...,...,...,...
1577,1578,601,822,82605,110726,0.731144,0.746031
1578,1579,816,1162,1322,1756,0.702238,0.752847
1579,1580,502,529,20434,20467,0.948960,0.998388
1580,1581,377,505,58266,142571,0.746535,0.408681


In [56]:
data = pd.merge(data, cust, how = 'left')
data.head(2)

,id,campaign_id,coupon_id,customer_id,redemption_status,flag,campaign_type,start_date,end_date,age_range,...,coup_brand_type_basket,ovr,over_set,perc_ovr,camp_cnt,cnt2,camp_qty,qty2,camp_shop_perc,camp_qty_perc
0,1,13,27,1053,0,Train,X,2013-05-19,2013-07-05,46-55,...,{Established},0,{},0.0,240,310,73380,105551,0.774194,0.695209
1,2,13,116,48,0,Train,X,2013-05-19,2013-07-05,36-45,...,{Local},0,{},0.0,228,385,309,12143,0.592208,0.025447


In [57]:

data.groupby('redemption_status')['camp_cnt'].describe(percentiles = [x/100 for x in list(range(10,100,10))]).reset_index()


,redemption_status,count,mean,std,min,10%,20%,30%,40%,50%,60%,70%,80%,90%,max
0,-1,50226.0,856.076673,524.657803,45.0,295.0,419.0,536.0,642.0,736.0,857.0,1002.0,1218.0,1632.0,3207.0
1,0,77640.0,695.987931,468.422575,0.0,221.0,316.0,398.0,500.0,598.0,699.0,817.0,1000.0,1301.0,3207.0
2,1,729.0,978.149520,491.903916,101.0,445.0,545.0,685.0,756.0,892.0,981.0,1158.0,1354.0,1775.0,2676.0


In [58]:

data.groupby('redemption_status')['camp_qty'].describe(percentiles = [x/100 for x in list(range(10,100,10))]).reset_index()


,redemption_status,count,mean,std,min,10%,20%,30%,40%,50%,60%,70%,80%,90%,max
0,-1,50226.0,120385.813901,188986.624604,71.0,575.0,928.0,1637.0,13030.0,32510.0,65530.0,127359.0,211921.0,385396.0,1904086.0
1,0,77640.0,98212.936721,172482.166258,0.0,372.0,653.0,1071.0,6347.0,18270.0,45628.0,91907.0,167184.0,308448.0,1904086.0
2,1,729.0,185209.802469,242105.340564,147.0,1191.0,11302.4,21457.6,47680.0,106211.0,154403.0,240453.0,331494.0,460703.0,1904086.0


In [59]:

data.groupby('redemption_status')['camp_qty_perc'].describe(percentiles = [x/100 for x in list(range(10,100,10))]).reset_index()


,redemption_status,count,mean,std,min,10%,20%,30%,40%,50%,60%,70%,80%,90%,max
0,-1,50226.0,0.714011,0.226100,0.002079,0.421281,0.575279,0.656469,0.693850,0.732429,0.784754,0.845226,0.924845,0.986803,1.000000
1,0,77640.0,0.678438,0.242665,0.000000,0.344949,0.524449,0.597315,0.666895,0.705751,0.751927,0.818158,0.897987,0.981080,0.999842
2,1,729.0,0.720994,0.208426,0.012899,0.421144,0.567861,0.651806,0.703513,0.732429,0.784991,0.850705,0.898096,0.981854,0.999822


### Transaction Info - 3.0

In [60]:
cust_t = pd.read_csv('data/train_AUpWtIz/customer_transaction_data.csv')

cust_t['sales'] = cust_t['quantity'] * cust_t['selling_price']

def f(row):
    if row['coupon_discount'] == 0:
        val = 0
    else: 
        val = 1
    return val

cust_t['coupon_disc_binary'] = cust_t.apply(f, axis =1)

cust = cust_t.groupby(['customer_id']).agg(qty = ('quantity', 'mean'),
                                           cnt = ('quantity', 'count'),
                                           price = ('selling_price', 'mean'),
                                           sales = ('sales', 'mean'),    
                                           disc = ('other_discount', 'mean'),
                                           coup_disc = ('coupon_discount', 'mean'),
                                           sum_coupon_binary = ('coupon_disc_binary', 'sum'),
                                           cnt_coupon_binary = ('coupon_disc_binary', 'count')
                                          ).reset_index()

cust['coupon_use_perc'] = cust['sum_coupon_binary']/cust['cnt_coupon_binary']

cust.head(2)

data = pd.merge(data, cust, how = 'left')
print(data.shape)
data.head(2)

(128595, 53)


,id,campaign_id,coupon_id,customer_id,redemption_status,flag,campaign_type,start_date,end_date,age_range,...,camp_qty_perc,qty,cnt,price,sales,disc,coup_disc,sum_coupon_binary,cnt_coupon_binary,coupon_use_perc
0,1,13,27,1053,0,Train,X,2013-05-19,2013-07-05,46-55,...,0.695209,340.487097,310,184.260484,224818.629387,-33.168935,-0.287258,1,310,0.003226
1,2,13,116,48,0,Train,X,2013-05-19,2013-07-05,36-45,...,0.025447,31.540260,385,234.247013,32536.284026,-27.699169,-3.215039,12,385,0.031169


In [61]:
data.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'flag', 'campaign_type', 'start_date', 'end_date', 'age_range',
       'marital_status', 'rented', 'family_size', 'no_of_children',
       'income_bracket', 'campaign_len', 'camp_months', 'camp_start_day',
       'unique_items', 'unique_brands', 'brands', 'brand_types', 'categories',
       'cust_basket', 'cust_basket_size', 'cust_catg_basket',
       'cust_brand_basket', 'cust_brand_type_basket', 'established_perc',
       'established_qty_mean', 'coup_basket', 'coup_basket_size',
       'coup_catg_basket', 'coup_brand_basket', 'coup_brand_type_basket',
       'ovr', 'over_set', 'perc_ovr', 'camp_cnt', 'cnt2', 'camp_qty', 'qty2',
       'camp_shop_perc', 'camp_qty_perc', 'qty', 'cnt', 'price', 'sales',
       'disc', 'coup_disc', 'sum_coupon_binary', 'cnt_coupon_binary',
       'coupon_use_perc'],
      dtype='object')

# Model 

In [62]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
import pickle
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
import catboost as cgb
from bayes_opt import BayesianOptimization
import xgboost
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression

In [63]:
data['brands'] = data['brands'].astype(str)
data['coup_catg_basket'] = data['coup_catg_basket'].astype(str)
data['coup_brand_type_basket'] = data['coup_brand_type_basket'].astype(str)

In [64]:
raw_columns = ['perc_ovr','coupon_use_perc','camp_shop_perc','camp_qty_perc','established_perc']

# 'brands'

#---------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------


catg_columns = ['campaign_type','age_range','marital_status', 'rented', 
                'family_size', 'no_of_children','income_bracket','camp_months','camp_start_day',
                'brand_types','categories','brands','coup_catg_basket', 'coup_brand_type_basket']

data = pd.get_dummies(data, drop_first = True, columns = catg_columns)


#---------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------


# Standardize features to 0 mean and unit variance # mean = 0, sd = 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

cont_columns = ['campaign_len','unique_items','unique_brands','cust_basket_size','coup_basket_size',
                'qty', 'cnt','price', 'disc', 'coup_disc','sales','sum_coupon_binary','camp_cnt','camp_qty',
                'established_qty_mean','ovr'
               ]

data[cont_columns] = scaler.fit_transform(data[cont_columns])


#---------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------


drop_col = ['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
            'flag', 'start_date', 'end_date','cust_basket',
            'coup_basket', 'over_set','cnt_coupon_binary','cnt2','qty2',
            'cust_brand_type_basket'
           ]

t = 'Train'
X = data.query('flag == @t').drop(drop_col, axis = 1)
y = data.query('flag == @t')['redemption_status']

t = 'Test'
X_t= data.query('flag == @t').drop(drop_col, axis = 1)


#---------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------


print(X.shape, y.shape)
print(X_t.shape)

(78369, 556) (78369,)
(50226, 556)


In [65]:
drop_col_2 = ['cust_catg_basket','cust_brand_basket','coup_brand_basket']

X = X.drop(drop_col_2, axis = 1)
X_t = X_t.drop(drop_col_2, axis = 1)

In [66]:
cat_feat = [x for x in list(X.columns) if x not in list(cont_columns + raw_columns)]
print(len(cat_feat))

532


In [67]:
# hold different models in a single dictionary
models = {}
models["Linear"]        = LinearRegression(fit_intercept=False)

models["XGBoost"]       = xgboost.XGBClassifier(n_estimators=1000, n_jobs=7, learning_rate=0.05, random_state = 8)

models["RandomForest"]  = RandomForestClassifier(n_estimators=1000, n_jobs=7, class_weight = 'balanced', 
                                                 random_state = 8)

models["LGBM"]          = lgb.sklearn.LGBMClassifier(class_weight = 'balanced', learning_rate=0.01, n_jobs = 7,
                                                     n_estimators = 1000, objective = 'binary', random_state = 8)

models["CatBoost"]      = cgb.CatBoostClassifier(iterations = 3000, learning_rate = 0.05, eval_metric = 'AUC', 
                                                 silent=True, random_state = 8, class_weights = [0.01,0.99],
                                                 cat_features = cat_feat)

In [68]:
model_name = 'XGBoost'
model   = models[model_name]
model.fit(X, y)
y_xb = model.predict_proba(X_t)[:,1]

In [69]:
model_name = 'CatBoost'
model   = models[model_name]
model.fit(X, y)
y_cb = model.predict_proba(X_t)[:,1]

In [70]:
model_name = 'LGBM'
model   = models[model_name]
model.fit(X, y)
y_lb = model.predict_proba(X_t)[:,1]

In [71]:
model_name = 'RandomForest'
model   = models[model_name]
model.fit(X, y)
y_rf = model.predict_proba(X_t)[:,1]

# Submission

In [76]:
coeff = [0.42679334, 0.04340608, 0.02087667, 0.5443711]
# coeff = [0.49194544, 0 ,0, 0.62173975]

In [77]:
y_hat = coeff[0]*y_xb + coeff[1]*y_cb + coeff[2]*y_lb + coeff[3]*y_rf

In [78]:
sub = pd.read_csv('data/sample_submission_Byiv0dS.csv')
sub['redemption_status'] = y_hat
sub.head()

,id,redemption_status
0,3,0.521679
1,4,0.003106
2,5,0.002548
3,8,0.000001
4,10,0.000002


In [79]:
sub.to_csv('2m.csv', index =False)